# 0. Importing Necessary Packages

In [1]:
# Printing the information of Python, IPython, OS, and the generation date.
%load_ext version_information
%version_information

Software versions
Python 3.7.9 64bit [GCC 7.3.0]
IPython 7.16.1
OS Linux 5.8.18 100.fc31.x86_64 x86_64 with fedora 31 Thirty_One
Mon Mar 28 16:17:14 2022 KST

In [26]:
# Printing the versions of packages
from importlib_metadata import version
for pkg in ['numpy', 'scipy', 'matplotlib', 'astropy', 'pandas', 'statsmodels']:
    print(pkg+": ver "+version(pkg))

numpy: ver 1.21.0
scipy: ver 1.6.2
matplotlib: ver 3.2.2
astropy: ver 4.2.1
pandas: ver 1.2.4
statsmodels: ver 0.12.2


In [27]:
# matplotlib backend
%matplotlib notebook

# importing necessary modules
import numpy as np
import glob, os
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
from matplotlib import pyplot as plt

# 1. Reading the Data

In [28]:
obs = pd.read_csv("Calibration/Observation.csv")
obs.head(10)

,Star,r_obs,airmass
0,1,10.484,1.199
1,2,11.641,1.199
2,3,9.215,1.199
3,4,9.047,1.199
4,5,9.502,1.199
5,6,10.025,1.199
6,7,9.079,1.199
7,8,9.914,1.199
8,9,13.015,1.331
9,10,13.206,1.331


In [29]:
lan = pd.read_csv("Calibration/Landolt_catalog.csv")
lan.head(10)

,Star,V,B-V,U-B,V-R,R-I,V-I
0,1,13.004,1.040,0.737,0.607,0.681,1.287
1,2,14.196,1.052,0.380,0.606,0.597,1.203
2,3,11.737,0.987,0.639,0.600,0.674,1.273
3,4,12.330,2.326,2.326,1.373,1.250,2.625
4,5,11.773,0.671,0.506,0.373,0.436,0.808
5,6,14.022,1.248,1.323,0.797,0.683,1.482
6,7,11.312,0.568,0.059,0.335,0.312,0.652
7,8,12.440,1.141,0.830,0.633,0.579,1.206
8,9,14.767,-0.252,-1.091,-0.111,-0.182,-0.296
9,10,15.553,0.830,0.356,0.433,0.389,0.824


In [ ]:
df = join